## libraries and function 

In [1]:
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
import pandas as pd
import time 
# from missingpy import MissForest
import math

ModuleNotFoundError: No module named 'sklearn.neighbors.base'

RESI

In [2]:
def compute_attribute_weights(A):
  from sklearn.preprocessing import minmax_scale
  y = minmax_scale(A, axis = 0)
  p = y.copy()
  ysum = y.sum(axis = 0)
  for i in range(y.shape[1]):
    if (ysum[i] == 0):
      p[:, i] = 1
    else:
      p[:, i] = y[:, i] / ysum[i]
  from scipy.special import xlogy
  E = - xlogy(p,p).sum(axis=0) / math.log(p.shape[0])
  w = (1 - E)/(len(E) - E.sum())
  if np.isnan(w).any():
    return None
  else:
    return w                                            
def generate_tuple_partition(CT, ICT, m):
  w = compute_attribute_weights(CT)
  r = np.ones(ICT.shape[0])
  for i in range(ICT.shape[0]):
    for j in range(ICT.shape[1]):
      if np.isnan(ICT[i,j]):
        r[i] -= w[j] #If NoneType then insufficient CT set has been used
  ICT = ICT[r.argsort()[::-1],:]
  return np.array_split(ICT, m)
def resi(Xtr_nan, m, n_neighbors):
  CT = [Xtr_nan[~np.isnan(Xtr_nan).any(axis=1)]]
  Tp = []
  T = generate_tuple_partition(CT[0],Xtr_nan[np.isnan(Xtr_nan).any(axis=1)], m)
  from sklearn.impute import KNNImputer
  for i in range(m):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputer.fit(CT[-1])
    Tp.append(imputer.transform(T[i]))
    CT.append(np.concatenate((CT[-1],Tp[-1])))
  Tpp = []
  for i in range(m):
    train = CT[0]
    for j in range(1,m):
      if j != i:
        train = np.concatenate((train,T[j]))
    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputer.fit(train)
    Tpp.append(imputer.transform(T[i]))
  imputer = KNNImputer(n_neighbors=n_neighbors)
  imputer.fit(CT[0])
  Tpp.append(imputer.transform(T[m-1]))
  CT = [CT[0]]
  for i in range(m):
    CT.append(np.concatenate((CT[-1],np.mean(np.array([Tp[i], Tpp[i]]), axis=0 ))))
  return CT[-1]

### compute_err function 

In [3]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  
 
def generate_nan(Xtrain,ytr, missing_rate):
  minimum_complete_samples = 3
  ct_id = np.random.choice(range(Xtrain.shape[0]), size = minimum_complete_samples, replace = False)
  CT = Xtrain[ct_id]
  ICT = Xtrain[[i for i in range(Xtrain.shape[0]) if i not in ct_id]]
  ICTshape = ICT.shape
  na_id = np.random.randint(0,ICT.size, round(missing_rate*ICT.size))
  ICT = ICT.flatten()
  ICT[na_id] = np.nan
  xxx = np.concatenate((CT,ICT.reshape(ICTshape)))
  ytrain = np.hstack((ytr[ct_id], np.array([ytr[i] for i in range(Xtrain.shape[0]) if i not in ct_id])))
  return xxx, ytrain
 
def compute_err_resi(Xtrain, ytrain, G, missing_rate, runs = 10):  
  e_rate = []
  for i in range(runs):
    Xtr_nan, ytrain = generate_nan(Xtrain, ytrain, missing_rate)
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    #RESI approach
    start = time.time()
    k = round(math.sqrt(len(np.unique(ytrain))))
    if (k%2 == 0):
      k += 1
    Xtr_resi = resi(Xtr_nan, 3, k) #Parameters: (Dataset, m, n_neighbors) 
    mus_resi = np.asarray([np.mean(Xtr_resi[ytrain==g,:], axis=0) for g in np.arange(G)])
    S_resi = np.asarray([(sum(ytrain==g))*np.cov(Xtr_resi[ytrain==g,:], rowvar =False)
             for g in np.arange(G)]) 
    resi_err = err(mus, S, mus_resi, S_resi)
    resi_time = time.time()-start
    e_rate.append(resi_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)

# Heart

In [4]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header = None,sep=',')
print(data.head())
test = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.test',
                     header=None, sep = ',')
data = pd.concat([data, test])
data = data.to_numpy()
X,y = data[:,1:], data[:,0]
X = X.astype(np.float32)
G = len(np.unique(y)) 
print(np.shape(X))
for g in range(G):
  print(sum(y==g))

   0   1   2   3   4   5   6   7   8   9   ...  35  36  37  38  39  40  41  \
0   1  59  52  70  67  73  66  72  61  58  ...  66  56  62  56  72  62  74   
1   1  72  62  69  67  78  82  74  65  69  ...  65  71  63  60  69  73  67   
2   1  71  62  70  64  67  64  79  65  70  ...  73  70  66  65  64  55  61   
3   1  69  71  70  78  61  63  67  65  59  ...  61  61  66  65  72  73  68   
4   1  70  66  61  66  61  58  69  69  72  ...  67  69  70  66  70  64  60   

   42  43  44  
0  74  64  67  
1  71  56  58  
2  41  51  46  
3  68  59  63  
4  55  49  41  

[5 rows x 45 columns]
(267, 44)
55
212


In [6]:
import math
resi_err = np.array([compute_err_resi(X, y, G, .2, runs = 10),
                    compute_err_resi(X, y, G, .35, runs = 10),
                    compute_err_resi(X, y, G, .5, runs = 10),
                    compute_err_resi(X, y, G, .65, runs = 10),
                    compute_err_resi(X, y, G, .8, runs = 10)])
resi_err.round(3)

array([[0.432, 0.021],
       [0.389, 0.018],
       [0.346, 0.015],
       [0.3  , 0.026],
       [0.264, 0.022]])


# Inosphere

In [7]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
# print(data.head())
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
print(len(y))
X = np.delete(X,[0,1], axis = 1)
print(X.shape)
print(np.linalg.matrix_rank(X))

351
(351, 32)
32


In [8]:
G = 2
resi_err = np.array([compute_err_resi(X, y, G, .2, runs = 10),
                    compute_err_resi(X, y, G, .35, runs = 10),
                    compute_err_resi(X, y, G, .5, runs = 10),
                    compute_err_resi(X, y, G, .65, runs = 10),
                    compute_err_resi(X, y, G, .8, runs = 10)])
resi_err.round(3)

array([[0.666, 0.013],
       [0.627, 0.019],
       [0.579, 0.026],
       [0.556, 0.03 ],
       [0.52 , 0.061]])

# seeds 

In [9]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
                     sep = '\s+', header = None)
data = pd.DataFrame.to_numpy(data)
X,y = data[:,:7], data[:,7]-1 # reset the labels to go start from 0  
print(X.shape)
print(np.linalg.matrix_rank(X))

(210, 7)
7


In [10]:
G = 3
resi_err = np.array([compute_err_resi(X, y, G, .2, runs = 10),
                    compute_err_resi(X, y, G, .35, runs = 10),
                    compute_err_resi(X, y, G, .5, runs = 10),
                    compute_err_resi(X, y, G, .65, runs = 10),
                    compute_err_resi(X, y, G, .8, runs = 10)])
resi_err.round(3)

array([[2.207, 0.035],
       [2.122, 0.051],
       [1.971, 0.054],
       [1.736, 0.11 ],
       [1.529, 0.118]])

# wine
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [11]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel()
print(X.shape)
print(np.linalg.matrix_rank(X))
# sum(y==0), sum(y==1), sum(y==2)

(178, 13)
13


In [12]:
G = 3
resi_err = np.array([compute_err_resi(X, y, G, .2, runs = 10),
                    compute_err_resi(X, y, G, .35, runs = 10),
                    compute_err_resi(X, y, G, .5, runs = 10),
                    compute_err_resi(X, y, G, .65, runs = 10),
                    compute_err_resi(X, y, G, .8, runs = 10)])
resi_err.round(3)

array([[0.619, 0.006],
       [0.566, 0.018],
       [0.499, 0.01 ],
       [0.429, 0.014],
       [0.38 , 0.011]])

# Iris
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [13]:
iris = datasets.load_iris()
X,y = iris.data, iris.target.ravel() 
G = 3
resi_err = np.array([compute_err_resi(X, y, G, .2, runs = 10),
                    compute_err_resi(X, y, G, .35, runs = 10),
                    compute_err_resi(X, y, G, .5, runs = 10),
                    compute_err_resi(X, y, G, .65, runs = 10),
                    compute_err_resi(X, y, G, .8, runs = 10)])
resi_err.round(3)

array([[2.67 , 0.086],
       [2.644, 0.083],
       [2.555, 0.106],
       [2.32 , 0.129],
       [2.121, 0.182]])